# SM Prototype 10K Train/Test Split, Vectorization, Linear Regression

In [63]:
import pandas as pd
import numpy as np
import re
import nltk
nltk.download('stopwords')
from utils import clean_text
import edgar
import pyLDAvis.sklearn
pyLDAvis.enable_notebook()

# Expand the max width of how our dataFrames display on screen
pd.options.display.max_colwidth = 500

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\yural\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
def ten_k_files():
    company_dict = {"FANG": "0001539838", "RSPP":"0001588216", "CXO":" 0001358071", "MDU": "0000067716",
               "LPI": "0001528129", "NFX": "0000912750", "ECA": "0001157806", "GPOR": "0000874499",
               "RRC": "0000315852", "EQT": "0000033213", "SM": "0000893538"}
    docs_df = pd.DataFrame(columns=['Filing_Type','Filing_Date','Company','Text','Label'])
    for comp in company_dict:
        year = 2019
        cik = company_dict[comp]
        company = edgar.Company(comp, cik)
        tree = company.getAllFilings(filingType = "10-K")
        docs = edgar.getDocuments(tree, noOfDocuments=5)

        for doc in docs:
            #print(comp)
            year -= 1
            
            docs_df = docs_df.append({'Filing_Type': '10-K','Filing_Date': year,'Company':comp, 'Text' :doc,'Label' :'1'},ignore_index=True)
    return docs_df



In [3]:
docs_df = ten_k_files()


In [5]:
docs_df['Filing_Date'] = docs_df['Filing_Date'].apply(str)

In [33]:
docs_df['Index'] = docs_df[['Filing_Date', 'Company']].apply(lambda x: ''.join(x), axis=1)
index_df = docs_df['Index']
index_df.to_csv("index.csv")

docs_df = docs_df.set_index('Index')
docs_df

,Filing_Type,Filing_Date,Company,Text,Label
Index,,,,,
2018FANG,10-K,2018,FANG,"\n10-K/A\n1\ndiamondback201810-ka.htm\nDIAMONDBACK 10-K/A\n\n\n\n\t\n\t\t\n\t\t\n\t\tDocument\n\t\n\t\n UNITED STATESSECURITIES AND EXCHANGE COMMISSIONWashington, D.C. 20549 FORM 10-K/AAmendment No. 1 ýANNUAL REPORT UNDER SECTION 13 OR 15(d) OF THE SECURITIES EXCHANGE ACT OF 1934For the fiscal year ended December 31, 2018 ORoTRANSITION REPORT UNDER SECTION 13 OR 15(d) OF SECURITIES EXCHANGE ACT OF 1934Commission File Number 001-35700 Diamondback Energy, Inc.(Exact Name of Re...",1
2017FANG,10-K,2017,FANG,"\n10-K\n1\ndiamondback201810-k.htm\nDIAMONDBACK 10-K\n\n\n\n\t\n\t\t\n\t\t\n\t\tDocument\n\t\n\t\nTable of Contents UNITED STATESSECURITIES AND EXCHANGE COMMISSIONWashington, D.C. 20549 FORM 10-K ýANNUAL REPORT UNDER SECTION 13 OR 15(d) OF THE SECURITIES EXCHANGE ACT OF 1934For the fiscal year ended December 31, 2018 OR¨TRANSITION REPORT UNDER SECTION 13 OR 15(d) OF SECURITIES EXCHANGE ACT OF 1934Commission File Number 001-35700 Diamondback Energy, Inc.(Exact Name of Registr...",1
2016FANG,10-K,2016,FANG,"\n10-K\n1\ndiamondback201710-k.htm\nDIAMONDBACK 10-K\n\n\n\n\t\n\t\t\n\t\t\n\t\tDocument\n\t\n\t\nTable of Contents UNITED STATESSECURITIES AND EXCHANGE COMMISSIONWashington, D.C. 20549 FORM 10-K ýANNUAL REPORT UNDER SECTION 13 OR 15(d) OF THE SECURITIES EXCHANGE ACT OF 1934For the fiscal year ended December 31, 2017 OR¨TRANSITION REPORT UNDER SECTION 13 OR 15(d) OF SECURITIES EXCHANGE ACT OF 1934Commission File Number 001-35700 Diamondback Energy, Inc.(Exact Name of Registr...",1
2015FANG,10-K,2015,FANG,"\n10-K\n1\ndiamondback201610-k.htm\nDIAMONDBACK 10-K\n\n\n\n\t\n\t\t\n\t\t\n\t\tDocument\n\t\n\t\nTable of Contents UNITED STATESSECURITIES AND EXCHANGE COMMISSIONWashington, D.C. 20549 FORM 10-K ýANNUAL REPORT UNDER SECTION 13 OR 15(d) OF THE SECURITIES EXCHANGE ACT OF 1934For the fiscal year ended December 31, 2016 OR¨TRANSITION REPORT UNDER SECTION 13 OR 15(d) OF SECURITIES EXCHANGE ACT OF 1934Commission File Number 001-35700 Diamondback Energy, Inc.(Exact Name of Registr...",1
2014FANG,10-K,2014,FANG,"\n10-K\n1\ndiamondback201510-k.htm\nDIAMONDBACK 10-K\n\n\n\n\t\n\t\t\n\t\t\n\t\t10-K\n\t\n\t\nTable of Contents UNITED STATESSECURITIES AND EXCHANGE COMMISSIONWashington, D.C. 20549 FORM 10-K ýANNUAL REPORT UNDER SECTION 13 OR 15(d) OF THE SECURITIES EXCHANGE ACT OF 1934For the fiscal year ended December 31, 2015 OR¨TRANSITION REPORT UNDER SECTION 13 OR 15(d) OF SECURITIES EXCHANGE ACT OF 1934Commission File Number 001-35700 Diamondback Energy, Inc.(Exact Name of Registrant ...",1
2018RSPP,10-K,2018,RSPP,"\n10-K/A\n1\nd580186d10ka.htm\nFORM 10-K/A\n\n\nForm 10-K/A\n\n \nTable of Contents\n\n UNITED STATES \nSECURITIES AND EXCHANGE COMMISSION \nWashington, D.C. 20549 \n FORM\n10-K/A (Amendment No. 1) \n \n(Mark one) ☒\n ANNUAL REPORT PURSUANT TO SECTION 13 OR 15(d) OF THE SECURITIES EXCHANGE ACT OF 1934 For the fiscal year\nended December 31, 2017 or \n☐ TRANSITION REPORT PURSUANT TO SECTION 13 OR 15(d) OF THE SECURITIES EXCHANGE ACT OF 1934 \nFor the transition period from\n ...",1
2017RSPP,10-K,2017,RSPP,"\n10-K\n1\na2017123110k.htm\nFORM 10-K\n\n\n\n\t\n\t\t\n\t\t\n\t\tDocument\n\t\n\t\nTable of ContentsUNITED STATESSECURITIES AND EXCHANGE COMMISSIONWashington, D.C. 20549 FORM 10-K (Mark one) ý ANNUAL REPORT PURSUANT TO SECTION 13 OR 15(d) OF THE SECURITIES EXCHANGE ACT OF 1934 For the fiscal year ended December 31, 2017 or o TRANSITION REPORT PURSUANT TO SECTION 13 OR 15(d) OF THE SECURITIES EXCHANGE ACT OF 1934 For the transition period from to Co...",1
2016RSPP,10-K,2016,RSPP,"\n10-K\n1\nrspp2016123110k.htm\n10-K\n\n\n\n\t\n\t\t\n\t\t\n\t\tDocument\n\t\n\t\nTable of ContentsUNITED STATESSECURITIES AND EXCHANGE COMMISSIONWashington, D.C. 20549 FORM 10-K (Mark one) ý ANNUAL REPORT PURSUANT TO SECTION 13 OR 15(d) OF THE SECURITIES EXCHANGE ACT OF 1934 For the fiscal year ended December 31, 2016 or o TRANSITION REPORT PURSUANT TO SECTION 13 OR

In [7]:
# Vectorization methods
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

# Classification model
from sklearn.linear_model import LinearRegression

### Train/Test Split

Train will be a 80% split of the corpus of documents comprising of 10K and 10Qs for the companies spanning time.
Test will be the 20% remaining documents.

Documents will be the features, and Labels will be the stock price change label (positive or negative)

There will be 3 main models: one for the stock change at 1 day, one for the stock change at 1 month, one for the stock change at 3 months (before the next quarter)

In [8]:
#first pass with just SM 10-K documents only
#company = edgar.Company("SM Energy", "0000893538")
#tree = company.getAllFilings(filingType = "10-K")
#docs = edgar.getDocuments(tree, noOfDocuments=6) #2014, 2015, 2016, 2017, 2018

In [9]:
#text2018 = docs[0]
#text2017 = docs[1]
#text2016 = docs[2]
#text2015 = docs[3]
#text2014 = docs[4]

In [28]:
# Create dataframe from data
#docs_df = pd.DataFrame(columns=['Filing_Type','Filing_Date','Company','Text','Label'])
#docs_df
#docs_df = docs_df.append({'Filing_Type': '10-K','Filing_Date': 2018,'Company':'SM', 'Text' :text2018,'Label' :'1'},ignore_index=True)
#docs_df = docs_df.append({'Filing_Type': '10-K','Filing_Date': 2017,'Company':'SM', 'Text' :text2017,'Label' :'0'},ignore_index=True)
#docs_df = docs_df.append({'Filing_Type': '10-K','Filing_Date': 2016,'Company':'SM', 'Text' :text2016,'Label' :'1'},ignore_index=True)
#docs_df = docs_df.append({'Filing_Type': '10-K','Filing_Date': 2015,'Company':'SM', 'Text' :text2015,'Label' :'0'},ignore_index=True)
#docs_df = docs_df.append({'Filing_Type': '10-K','Filing_Date': 2014,'Company':'SM', 'Text' :text2014,'Label' :'1'},ignore_index=True)

docs_df


,Filing_Type,Filing_Date,Company,Text,Label
Index,,,,,
2018FANG,10-K,2018,FANG,"\n10-K/A\n1\ndiamondback201810-ka.htm\nDIAMONDBACK 10-K/A\n\n\n\n\t\n\t\t\n\t\t\n\t\tDocument\n\t\n\t\n UNITED STATESSECURITIES AND EXCHANGE COMMISSIONWashington, D.C. 20549 FORM 10-K/AAmendment No. 1 ýANNUAL REPORT UNDER SECTION 13 OR 15(d) OF THE SECURITIES EXCHANGE ACT OF 1934For the fiscal year ended December 31, 2018 ORoTRANSITION REPORT UNDER SECTION 13 OR 15(d) OF SECURITIES EXCHANGE ACT OF 1934Commission File Number 001-35700 Diamondback Energy, Inc.(Exact Name of Re...",1
2017FANG,10-K,2017,FANG,"\n10-K\n1\ndiamondback201810-k.htm\nDIAMONDBACK 10-K\n\n\n\n\t\n\t\t\n\t\t\n\t\tDocument\n\t\n\t\nTable of Contents UNITED STATESSECURITIES AND EXCHANGE COMMISSIONWashington, D.C. 20549 FORM 10-K ýANNUAL REPORT UNDER SECTION 13 OR 15(d) OF THE SECURITIES EXCHANGE ACT OF 1934For the fiscal year ended December 31, 2018 OR¨TRANSITION REPORT UNDER SECTION 13 OR 15(d) OF SECURITIES EXCHANGE ACT OF 1934Commission File Number 001-35700 Diamondback Energy, Inc.(Exact Name of Registr...",1
2016FANG,10-K,2016,FANG,"\n10-K\n1\ndiamondback201710-k.htm\nDIAMONDBACK 10-K\n\n\n\n\t\n\t\t\n\t\t\n\t\tDocument\n\t\n\t\nTable of Contents UNITED STATESSECURITIES AND EXCHANGE COMMISSIONWashington, D.C. 20549 FORM 10-K ýANNUAL REPORT UNDER SECTION 13 OR 15(d) OF THE SECURITIES EXCHANGE ACT OF 1934For the fiscal year ended December 31, 2017 OR¨TRANSITION REPORT UNDER SECTION 13 OR 15(d) OF SECURITIES EXCHANGE ACT OF 1934Commission File Number 001-35700 Diamondback Energy, Inc.(Exact Name of Registr...",1
2015FANG,10-K,2015,FANG,"\n10-K\n1\ndiamondback201610-k.htm\nDIAMONDBACK 10-K\n\n\n\n\t\n\t\t\n\t\t\n\t\tDocument\n\t\n\t\nTable of Contents UNITED STATESSECURITIES AND EXCHANGE COMMISSIONWashington, D.C. 20549 FORM 10-K ýANNUAL REPORT UNDER SECTION 13 OR 15(d) OF THE SECURITIES EXCHANGE ACT OF 1934For the fiscal year ended December 31, 2016 OR¨TRANSITION REPORT UNDER SECTION 13 OR 15(d) OF SECURITIES EXCHANGE ACT OF 1934Commission File Number 001-35700 Diamondback Energy, Inc.(Exact Name of Registr...",1
2014FANG,10-K,2014,FANG,"\n10-K\n1\ndiamondback201510-k.htm\nDIAMONDBACK 10-K\n\n\n\n\t\n\t\t\n\t\t\n\t\t10-K\n\t\n\t\nTable of Contents UNITED STATESSECURITIES AND EXCHANGE COMMISSIONWashington, D.C. 20549 FORM 10-K ýANNUAL REPORT UNDER SECTION 13 OR 15(d) OF THE SECURITIES EXCHANGE ACT OF 1934For the fiscal year ended December 31, 2015 OR¨TRANSITION REPORT UNDER SECTION 13 OR 15(d) OF SECURITIES EXCHANGE ACT OF 1934Commission File Number 001-35700 Diamondback Energy, Inc.(Exact Name of Registrant ...",1
2018RSPP,10-K,2018,RSPP,"\n10-K/A\n1\nd580186d10ka.htm\nFORM 10-K/A\n\n\nForm 10-K/A\n\n \nTable of Contents\n\n UNITED STATES \nSECURITIES AND EXCHANGE COMMISSION \nWashington, D.C. 20549 \n FORM\n10-K/A (Amendment No. 1) \n \n(Mark one) ☒\n ANNUAL REPORT PURSUANT TO SECTION 13 OR 15(d) OF THE SECURITIES EXCHANGE ACT OF 1934 For the fiscal year\nended December 31, 2017 or \n☐ TRANSITION REPORT PURSUANT TO SECTION 13 OR 15(d) OF THE SECURITIES EXCHANGE ACT OF 1934 \nFor the transition period from\n ...",1
2017RSPP,10-K,2017,RSPP,"\n10-K\n1\na2017123110k.htm\nFORM 10-K\n\n\n\n\t\n\t\t\n\t\t\n\t\tDocument\n\t\n\t\nTable of ContentsUNITED STATESSECURITIES AND EXCHANGE COMMISSIONWashington, D.C. 20549 FORM 10-K (Mark one) ý ANNUAL REPORT PURSUANT TO SECTION 13 OR 15(d) OF THE SECURITIES EXCHANGE ACT OF 1934 For the fiscal year ended December 31, 2017 or o TRANSITION REPORT PURSUANT TO SECTION 13 OR 15(d) OF THE SECURITIES EXCHANGE ACT OF 1934 For the transition period from to Co...",1
2016RSPP,10-K,2016,RSPP,"\n10-K\n1\nrspp2016123110k.htm\n10-K\n\n\n\n\t\n\t\t\n\t\t\n\t\tDocument\n\t\n\t\nTable of ContentsUNITED STATESSECURITIES AND EXCHANGE COMMISSIONWashington, D.C. 20549 FORM 10-K (Mark one) ý ANNUAL REPORT PURSUANT TO SECTION 13 OR 15(d) OF THE SECURITIES EXCHANGE ACT OF 1934 For the fiscal year ended December 31, 2016 or o TRANSITION REPORT PURSUANT TO SECTION 13 OR

In [9]:
### Assign X (data) and y (target)
X = docs_df.drop("Label", axis=1)
y = docs_df["Label"]
print(X.shape, y.shape)

(53, 4) (53,)


In [10]:
#train/test split at 80%
from sklearn.model_selection import train_test_split
XTrain, XTest, yTrain, yTest = train_test_split(X, y, test_size = 0.2, random_state = 42)

In [11]:
#look at train/test split
XTrain
yTrain
XTest
yTest

Index
2014MDU     1
2015RRC     1
2014EQT     1
2016CXO     1
2018EQT     1
2018RSPP    1
2016MDU     1
2016SM      1
2015FANG    1
2016ECA     1
2015CXO     1
Name: Label, dtype: object

In [12]:
#Create train/test dataframes
n_docs = 100000 #number of times to run model
train_docs = XTrain
train_labels = yTrain
test_docs = XTest
test_labels = yTest

train_df = pd.DataFrame({"text": train_docs['Text'], "labels": train_labels})
train_df

test_df = pd.DataFrame({"text": test_docs['Text'], "labels": test_labels})
test_df

print(f"Train Shape: {train_df.shape}")
print(f"Test Shape: {test_df.shape}")

Train Shape: (42, 2)
Test Shape: (11, 2)


In [13]:
train_df

,text,labels
Index,,
2015RSPP,"\n10-K\n1\na2015123110k.htm\n10-K\n\n\n\n\t\n\t\t\n\t\t\n\t\t10-K\n\t\n\t\nTable of ContentsUNITED STATESSECURITIES AND EXCHANGE COMMISSIONWashington, D.C. 20549 FORM 10-K (Mark one) ý ANNUAL REPORT PURSUANT TO SECTION 13 OR 15(d) OF THE SECURITIES EXCHANGE ACT OF 1934 For the fiscal year ended December 31, 2015 or o TRANSITION REPORT PURSUANT TO SECTION 13 OR 15(d) OF THE SECURITIES EXCHANGE ACT OF 1934 For the transition period from to Commission ...",1
2017NFX,"\n10-K\n1\nnfx201610-k.htm\n10-K\n\n\n\n\t\n\t\t\n\t\t\n\t\tDocument\n\t\n\t\n UNITED STATES SECURITIES AND EXCHANGE COMMISSIONWashington, D.C. 20549Form 10-K þANNUAL REPORT PURSUANT TO SECTION 13 OR 15(d) OF THE SECURITIES EXCHANGE ACT OF 1934 For the fiscal year ended December 31, 2016or ¨TRANSITION REPORT PURSUANT TO SECTION 13 OR 15(d) OF THE SECURITIES EXCHANGE ACT OF 1934 For the transition period from to .Commission file number: 1-12534Newfield Explora...",1
2017RSPP,"\n10-K\n1\na2017123110k.htm\nFORM 10-K\n\n\n\n\t\n\t\t\n\t\t\n\t\tDocument\n\t\n\t\nTable of ContentsUNITED STATESSECURITIES AND EXCHANGE COMMISSIONWashington, D.C. 20549 FORM 10-K (Mark one) ý ANNUAL REPORT PURSUANT TO SECTION 13 OR 15(d) OF THE SECURITIES EXCHANGE ACT OF 1934 For the fiscal year ended December 31, 2017 or o TRANSITION REPORT PURSUANT TO SECTION 13 OR 15(d) OF THE SECURITIES EXCHANGE ACT OF 1934 For the transition period from to Co...",1
2017GPOR,"\n10-K\n1\ngpor-12312017x10k.htm\n10-K\n\n\n\n\t\n\t\t\n\t\t\n\t\tDocument\n\t\n\t\nTable of ContentsIndex to Financial StatementsUNITED STATESSECURITIES AND EXCHANGE COMMISSIONWashington, D.C. 20549 FORM 10-K (Mark One)ýANNUAL REPORT UNDER SECTION 13 OR 15(d) OF THE SECURITIES EXCHANGE ACT OF 1934 For the fiscal year ended December 31, 2017 OR¨TRANSITION REPORT UNDER SECTION 13 OR 15(d) OF SECURITIES EXCHANGE ACT OF 1934 For the transition period from to ...",1
2014FANG,"\n10-K\n1\ndiamondback201510-k.htm\nDIAMONDBACK 10-K\n\n\n\n\t\n\t\t\n\t\t\n\t\t10-K\n\t\n\t\nTable of Contents UNITED STATESSECURITIES AND EXCHANGE COMMISSIONWashington, D.C. 20549 FORM 10-K ýANNUAL REPORT UNDER SECTION 13 OR 15(d) OF THE SECURITIES EXCHANGE ACT OF 1934For the fiscal year ended December 31, 2015 OR¨TRANSITION REPORT UNDER SECTION 13 OR 15(d) OF SECURITIES EXCHANGE ACT OF 1934Commission File Number 001-35700 Diamondback Energy, Inc.(Exact Name of Registrant ...",1
2014GPOR,"\n10-K\n1\ngpor-12312014x10k.htm\n10-K\n\n\n\n\t\n\t\t\n\t\t\n\t\tGPOR-12.31.2014-10K\n\t\n\t\nTable of ContentsIndex to Financial StatementsUNITED STATESSECURITIES AND EXCHANGE COMMISSIONWashington, D.C. 20549 FORM 10-K (Mark One)ýANNUAL REPORT UNDER SECTION 13 OR 15(d) OF THE SECURITIES EXCHANGE ACT OF 1934 For the fiscal year ended December 31, 2014 OR¨TRANSITION REPORT UNDER SECTION 13 OR 15(d) OF SECURITIES EXCHANGE ACT OF 1934 For the transition period from to ...",1
2014LPI,"\n10-K\n1\na2014form10-k.htm\n10-K\n\n\n\n\t\n\t\t\n\t\t\n\t\t2014 Form 10-K\n\t\n\t\nUNITED STATESSECURITIES AND EXCHANGE COMMISSIONWashington, D.C. 20549FORM 10-Ký ANNUAL REPORT PURSUANT TO SECTION 13 OR 15(d) OF THE SECURITIES EXCHANGE ACT OF 1934For the fiscal year ended December 31, 2014oro TRANSITION REPORT PURSUANT TO SECTION 13 OR 15(d) OF THE SECURITIES EXCHANGE ACT OF 1934Commission file number: 001-35380Laredo Petroleum, Inc.(Exact name of registrant as specified in its charter)De...",1
2017EQT,"\n10-K\n1\neqt1231201810k.htm\n10-K\n\n\n\n\t\n\t\t\n\t\t\n\t\tDocument\n\t\n\t\nUNITED STATESSECURITIES AND EXCHANGE COMMISSIONWashington, D.C. 20549 FORM 10-K [X] ANNUAL REPORT PURSUANT TO SECTION 13 OR 15(d) OF THE SECURITIES EXCHANGE ACT OF 1934FOR THE FISCAL YEAR ENDED DECEMBER 31, 2018 [ ]TRANSITION REPORT PURSUANT TO SECTION 13 OR 15(d) OF THE SECURITIES EXCHANGE ACT OF 1934orFOR THE TRANSITION PERIOD FROM ___________ TO __________ COMMISSION FILE NUMBER 001-03551 EQT CORPORATION(E...",1
2018GPOR,"\n10-K\n1\ngpor-12312018x10k.htm\n10-K\n\n\n\n\t\n\t\t\n\t\t\n\t\tDo

## CountVectorization

Turn the raw text from TrainText and TestText into feature vectors so that we can use them in our model.
We vectorize the text in 2 steps: 
1. First, we `fit`, the training data to our vectorizer to compute the vocabulary (feature set). 
2. Then, we `transform` with our text for both train and test to count the number occurrences for each word in our vocabulary.

The output of the CountVectorizer's `transform` task is a [sparse matrix](https://docs.scipy.org/doc/scipy/reference/generated/scipy.sparse.csr_matrix.html#scipy.sparse.csr_matrix), which condenses the matrix values to avoid storing an excessive amount of zeros.

In [14]:
vectorizer = CountVectorizer(stop_words='english') ##could limit the words being used in here
vectorizer.fit(train_df['text']) ##fit training data's text to get the feature set
train_vecs = vectorizer.transform(train_df['text']) ##transform with training to count word frequencies
test_vecs = vectorizer.transform(test_df['text']) ##transform with test data to count word frequencies

In [20]:
all_df = pd.DataFrame({"text": docs_df['Text']})

all_vecs = vectorizer.transform(all_df['text']) ##transform with all data to count word frequencies


In [22]:
#full vocabulary size for training data
print(f"Number of documents: {train_vecs.shape[0]}")
print(f"Size of vocabulary: {train_vecs.shape[1]}")

#full vocabulary size for testing data
print(f"Number of documents: {test_vecs.shape[0]}")
print(f"Size of vocabulary: {test_vecs.shape[1]}")

#full vocabulary size for all data
print(f"Number of documents: {all_vecs.shape[0]}")
print(f"Size of vocabulary: {all_vecs.shape[1]}")

Number of documents: 42
Size of vocabulary: 29822
Number of documents: 11
Size of vocabulary: 29822
Number of documents: 53
Size of vocabulary: 29822


In [23]:
#get rid of non-zero values because it's a sparse matrix 
# Train
print(f"Number of TRAINING non-zero features: {train_vecs.nnz}")
print(f"Number of TRAINING zero features: {(train_vecs.shape[0]*train_vecs.shape[1])-train_vecs.nnz}")

# Test
print(f"Number of TEST non-zero features: {test_vecs.nnz}")
print(f"Number of TEST zero features: {(test_vecs.shape[0]*test_vecs.shape[1])-test_vecs.nnz}")

# All
print(f"Number of TEST non-zero features: {all_vecs.nnz}")
print(f"Number of TEST zero features: {(all_vecs.shape[0]*all_vecs.shape[1])-all_vecs.nnz}")

Number of TRAINING non-zero features: 195463
Number of TRAINING zero features: 1057061
Number of TEST non-zero features: 47247
Number of TEST zero features: 280795
Number of TEST non-zero features: 242710
Number of TEST zero features: 1337856


### Display all terms

Displays and exports all_vecs word frequencies to .csv

In [30]:

df_all_counts = pd.DataFrame(all_vecs.toarray(), 
                         columns=vectorizer.get_feature_names())[:100].T
df_all_counts.to_csv("all10kfinal2.csv")


## Term Frequency-Inverse Document Frequency (TF-IDF)

Tf-idf is a statistical representation of how relevant a word is to a particular document within a corpus. _Relevance_, in this scenario, can be defined as how much information a word provides about the context of one document vs all other documents in the corpus. 

In short, tf-idf is calculated by comparing the number of times that a particular terms occurs in a given document vs the number of other documents in the corpus that contain that word. A word that frequently occurs in 1 document, but only occurs in a very small number of other documents will have a high tf-idf score.

The calculation for tf-idf is the product of two smaller calculations:

$$TF_{i,j} = \frac{Number~of~times~word_{i}~occurs~in~document_{j}}{Total~number~of~words~in~document_{j}}$$


$$IDF_{i} = log(\frac{Total~number~of~documents~in~corpus}{Number~of~documents~that~contain~word_{i}})$$

This might actually be useful for us because we can see if specific terms are especially important to certain documents relative to others -- this only matters because maybe we can see when acquisitions might be on the horizon, for example.

In [19]:
#tf-idf vectorization using calculation
tfidf_vectorizer = TfidfVectorizer(stop_words='english')
tfidf_vectorizer.fit(train_df['text'])
train_tfidf_vecs = tfidf_vectorizer.transform(train_df['text'])
test_tfidf_vecs = tfidf_vectorizer.transform(test_df['text'])

### Display a few terms and their tf-idf scores for a few documents

This is only meant to be used for demonstration purposes. The cell below has no impact on the actual execution of our task. Also, this cell is only intended for use when the number of documents is small (<100), otherwise it will likely only display a bunch of zeros.

In [54]:
#alice note -- obviously need to clean this up a bit.
#df_tfidf = pd.DataFrame(train_tfidf_vecs.toarray(), 
                         #columns=tfidf_vectorizer.get_feature_names())[:15].T
#df_tfidf.tail(20)

#### Comparison of the representation of different words vs. straight up word frequency

In [51]:
#pd.DataFrame({"TF-IDF of stock":df_tfidf.loc['stock'], "CountVectorizer: Stock":df_counts.loc['stock']})

In [ ]:
#pd.DataFrame({"TF-IDF of drilling":df_tfidf.loc['drilling'], "CountVectorizer: Drilling":df_counts.loc['drilling']})

# LDA Modeling

uses bag of words model (after term-document matrix has already been created) - used as dimension reduction or noise reducing.

need to figure out optimum number of topics

In [34]:
import os.path
from gensim import corpora
from gensim.models import LsiModel
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from gensim.models.coherencemodel import CoherenceModel
import matplotlib.pyplot as plt

In [ ]:
all_10k_dict = vectorizer.get_feature_names()


In [73]:
#fit feature models to LDA Model
from sklearn.decomposition import LatentDirichletAllocation

no_topics = 5

lda = LatentDirichletAllocation(n_components=no_topics, random_state=4, evaluate_every=1).fit(all_vecs)

C:\Users\yural\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\decomposition\online_lda.py:536: DeprecationWarning: The default value for 'learning_method' will be changed from 'online' to 'batch' in the release 0.20. This warning was introduced in 0.18.
  DeprecationWarning)


In [60]:
def display_topics(model, feature_names, no_top_words):
    for topic_idx, topic in enumerate(model.components_):
        print(f"Topic: {topic_idx}")
        print(" ".join([feature_names[i]
                        for i in topic.argsort()[:-no_top_words - 1:-1]]))

no_top_words = 10
display_topics(lda, all_10k_dict, no_top_words)

Topic: 0
gas oil natural december 31 company financial production million net
Topic: 1
company 2018 executive compensation performance 2017 committee awards mr stock
Topic: 2
company december oil tagged 31 financial gas natural million 2016
Topic: 3
company gas 31 december oil million natural 2016 financial net
Topic: 4
gas natural company december financial 31 2016 million costs oil


In [65]:
pyLDAvis.sklearn.prepare(lda, all_vecs, vectorizer)

C:\Users\yural\AppData\Local\Continuum\anaconda3\lib\site-packages\pyLDAvis\_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
3      0.084375  0.002149       1        1  61.104344
0      0.082008 -0.028639       2        1  36.900618
1      0.006526  0.050380       3        1   1.868747
2     -0.072783 -0.020242       4        1   0.125809
4     -0.100127 -0.003648       5        1   0.000481, topic_info=      Category          Freq           Term         Total  loglift  logprob
term                                                                       
20507  Default  24489.000000            gas  24489.000000  30.0000  30.0000
16828  Default  22630.000000        company  22630.000000  29.0000  29.0000
18200  Default  19422.000000       december  19422.000000  28.0000  28.0000
23216  Default  18195.000000        natural  18195.000000  27.0000  27.0000
5824   Default  19344.000000             31  19344.000000  26.0000  26.0000
23733  Default  20901.000000            oil  20901.000000  25.0000  25.0000
20176  Default  13685.000000      financial  13685.000000  24.0000  24.0000
3758   Default  11807.000000           2016  11807.000000  23.0000  23.0000
22976  Default  15448.000000        million  15448.000000  22.0000  22.0000
17733  Default  10443.000000          costs  10443.000000  21.0000  21.0000
16366  Default  11084.000000           cash  11084.000000  20.0000  20.0000
29244  Default   8844.000000          value   8844.000000  19.0000  19.0000
26144  Default   9958.000000       reserves   9958.000000  18.0000  18.0000
3917   Default   8691.000000           2017   8691.000000  17.0000  17.0000
3564   Default  11291.000000           2015  11291.000000  16.0000  16.0000
20479  Default   9337.000000         future   9337.000000  15.0000  15.0000
29697  Default   9092.000000           year   9092.000000  14.0000  14.0000
25066  Default  12760.000000     production  12760.000000  13.0000  13.0000
15418  Default   8607.000000         assets   8607.000000  12.0000  12.0000
23263  Default  12471.000000            net  12471.000000  11.0000  11.0000
1155   Default  10384.000000             10  10384.000000  10.0000  10.0000
19267  Default   8643.000000          ended   8643.000000   9.0000   9.0000
23821  Default  10925.000000     operations  10925.000000   8.0000   8.0000
27905  Default   8066.000000          stock   8066.000000   7.0000   7.0000
3433   Default  10131.000000           2014  10131.000000   6.0000   6.0000
28359  Default   8173.000000            tax   8173.000000   5.0000   5.0000
24896  Default   7837.000000          price   7837.000000   4.0000   4.0000
21227  Default   8192.000000         income   8192.000000   3.0000   3.0000
25313  Default   6905.000000         proved   6905.000000   2.0000   2.0000
4020   Default   5568.000000           2018   5568.000000   1.0000   1.0000
...        ...           ...            ...           ...      ...      ...
17733   Topic5      0.023979          costs  10443.224192  -0.7397  -6.1018
29244   Topic5      0.021663          value   8844.453982  -0.6751  -6.2033
26144   Topic5      0.022756       reserves   9958.767112  -0.7445  -6.1541
15418   Topic5      0.020619         assets   8607.286429  -0.6973  -6.2527
16828   Topic5      0.033986        company  22630.651307  -1.1643  -5.7530
5824    Topic5      0.030301             31  19344.250017  -1.1221  -5.8678
20479   Topic5      0.019835         future   9337.147929  -0.8175  -6.2915
22976   Topic5      0.025538        million  15448.215065  -1.0682  -6.0388
16366   Topic5      0.021380           cash  11084.527793  -0.9140  -6.2165
3917    Topic5      0.018857           2017   8691.740987  -0.7964  -6.3421
29697   Topic5      0.019162           year   9092.418552  -0.8254  -6.3260
19267   Topic5      0.018637          ended   8643.601406  -0.8026  -6.3538
3564    Topic5      0.021270           2015  11291.689173  -0.9377  -6.2217
20044   Topic5      0.014990           fair   5834.148082  -0.6272  -6.5715
15298  

In [72]:
lda_output = lda.transform(all_vecs)

# column names
topicnames = ["Topic" + str(i) for i in range(no_topics)]

# index names
docnames = ["Doc" + str(i) for i in range(len(all_df))]

# Make the pandas dataframe
df_document_topic = pd.DataFrame(np.round(lda_output, 2), columns=topicnames, index=docnames)

# Get dominant topic for each document
dominant_topic = np.argmax(df_document_topic.values, axis=1)
df_document_topic['dominant_topic'] = dominant_topic

# Styling
def color_green(val):
    color = 'green' if val > .1 else 'black'
    return 'color: {col}'.format(col=color)

def make_bold(val):
    weight = 700 if val > .1 else 400
    return 'font-weight: {weight}'.format(weight=weight)

# Apply Style
df_document_topics = df_document_topic.style.applymap(color_green).applymap(make_bold)
df_document_topics

,Topic0,Topic1,Topic2,Topic3,Topic4,dominant_topic
Doc0,0.44,0.56,0,0,0,1
Doc1,0.99,0,0,0.01,0,0
Doc2,1,0,0,0,0,0
Doc3,1,0,0,0,0,0
Doc4,1,0,0,0,0,0
Doc5,0,1,0,0,0,1
Doc6,0.76,0,0,0.24,0,0
Doc7,0.91,0,0,0.09,0,0
Doc8,0.97,0,0,0.03,0,0
Doc9,0.98,0,0,0.02,0,0


# Modeling -- classifying whether certain terms appearing will result in a positive or negative stock price

Vectorizing our data has converted our text data into a numeric feature set. Using these vectors, we can now begin to develop machine learning models for things like classification.

To further this model, look into better preprocessing, regression regularization, vocabulary pruning for feature selection, and hyperparameter tuning.

In [23]:
#determine regression model
from sklearn.linear_model import LinearRegression
classifier = LinearRegression()
classifier

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [24]:
classifier.fit(train_vecs, train_df['labels'])
predictions = classifier.predict(test_vecs)
print(f"Prediction   {predictions[:10]}")
print(f"Actual labels: {test_df['labels']}")

The exact solution is  x = 0                              
Prediction   [1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
Actual labels: 19    1
41    1
47    1
12    1
43    1
5     1
17    1
50    1
3     1
32    1
13    1
Name: labels, dtype: object


In [25]:
#Run a linear regression classification on the count vectors
count_linearReg = LinearRegression()
count_linearReg.fit(train_vecs, train_df['labels']) 
#train_vecs = frequency/ terms (unique features for this document)
#['labels'] is labeling if the change in stock price was positive or negative for the time frame
count_preds = count_linearReg.predict(test_vecs) 
#model.coef_ -> gives you coefficient values for all of terms. then look to see which have high weights / low weights. 
#terms are being weighted off of the change in the stock price. 
#if you see term in high weight (occurrence results in stock going up), (if low / negative weight, stock going down)

# Calculate the percentage of accurate predictions
accuracy = np.mean(count_preds==test_df['labels'])
print(f"LinearReg CountVectorizer accuracy: {accuracy}") ##linear regression, fit with the frequency*weights, 

The exact solution is  x = 0                              
LinearReg CountVectorizer accuracy: 0.0


### Run a logistic regression classification on the TF-IDF vectors

In [26]:
tfidf_linReg = LinearRegression()
tfidf_linReg.fit(train_tfidf_vecs, train_df['labels'])
tfidf_preds = tfidf_linReg.predict(test_tfidf_vecs)

# Calculate the percentage of accurate predictions
accuracy = np.mean(tfidf_preds==test_df['labels'])
print(f"LinReg TF-IDF accuracy: {accuracy}")

The exact solution is  x = 0                              
LinReg TF-IDF accuracy: 0.0


### View the terms with the highest coefficient values for each category

Notice that the terms highly weighted for each category seem to have highly negative weights for other categories. If we were to use more similarly related categories, we may not see such drastic differences.

Ignore the code behind this table. It is poorly written, but demonstrates the correct results.

In [67]:
from utils import getTopCoefs

#getTopCoefs(num_terms=5, model=tfidf_linReg, class_labels=train_df['labels'], feature_names=tfidf_vectorizer.get_feature_names())

### View coefficient weights for CountVectorizer features

In [66]:
#getTopCoefs(num_terms=5, model=count_linearReg, class_labels=train_df['labels'], feature_names=vectorizer.get_feature_names())

In [27]:
#Predicted inaccurately:
# Expand the max width of how our dataFrames display on screen
pd.options.display.max_colwidth = 1000

# Compile a dataframe with our text, the actual label, and the predicted label
final_df = pd.DataFrame({"text": test_df['text'], "Actual": test_df['labels'], "Prediction": tfidf_preds})

# Display the rows of our dataframe where the actual label and predicted label don't match
final_df.loc[(final_df['Actual'] != final_df['Prediction'])]

,text,Actual,Prediction
19,"\n10-K\n1\nmdu-12312014form10xk.htm\nMDU RESOURCES 2014 FORM 10-K\n\n\n\n\t\n\t\t\n\t\t\n\t\tWdesk | MDU-12.31.2014 Form 10-K\n\t\n\t\nUNITED STATES SECURITIES AND EXCHANGE COMMISSIONWASHINGTON, D.C. 20549FORM 10-Ký ANNUAL REPORT PURSUANT TO SECTION 13 OR 15(d) OF THE SECURITIES EXCHANGE ACT OF 1934For the fiscal year ended December 31, 2014ORo TRANSITION REPORT PURSUANT TO SECTION 13 OR 15(d) OF THE SECURITIES EXCHANGE ACT OF 1934For the transition period from _____________ to ______________Commission file number 1-3480MDU RESOURCES GROUP, INC.(Exact name of registrant as specified in its charter)Delaware 41-0423660(State or other jurisdiction ofincorporation or organization) (I.R.S. Employer Identification No.)1200 West Century AvenueP.O. Box 5650Bismarck, North Dakota 58506-5650(Address of principal executive offices)(Zip Code)(701) 530-1000(Registrant's telephone number, including area code)Securities registered pursuant to Section 12(b) of the Act:Title of each class Name of e...",1,1.0
41,"\n10-K\n1\nrrc-10k_20151231.htm\n10-K\n\n\n\n\n\nrrc-10k_20151231.htm\n\n\n\n\n\n \n \nUNITED STATES \nSECURITIES AND EXCHANGE COMMISSION \nWASHINGTON, D.C. 20549 \n \nFORM 10-K \n \n(Mark one) \n\n\n\n\nx\n\nANNUAL REPORT PURSUANT TO SECTION 13 OR 15(d) OF THE SECURITIES EXCHANGE ACT OF 1934 \nFor the fiscal year ended December 31, 2015\nOR \n\n\n\n\n¨\n\nTRANSITION REPORT PURSUANT TO SECTION 13 OR 15(d) OF THE SECURITIES EXCHANGE ACT OF 1934 \nFor the transition period from to \nCommission File Number: 001-12209 \n \nRANGE RESOURCES CORPORATION \n(Exact Name of Registrant as Specified in Its Charter) \n \n \n\n\n\n\nDelaware\n\n \n\n34-1312571\n\n\n\n(State or Other Jurisdiction of Incorporation or Organization)\n\n \n\n(IRS Employer Identification No.)\n\n\n\n \n\n \n\n\n\n100 Throckmorton Street, Suite 1200, Fort Worth, Texas\n\n \n\n76102\n\n\n\n(Address of Principal Executive Offices)\n\n \n\n(Zip Code)\n\n\nRegistrant’s telephone number, including area...",1,1.0
47,"\n10-K\n1\neqt-12312015x10k.htm\n10-K\n\n\n\n\t\n\t\t\n\t\t\n\t\t10-K\n\t\n\t\nTable of ContentsUNITED STATESSECURITIES AND EXCHANGE COMMISSIONWashington, D.C. 20549 FORM 10-K [X] ANNUAL REPORT PURSUANT TO SECTION 13 OR 15(d) OF THE SECURITIES EXCHANGE ACT OF 1934FOR THE FISCAL YEAR ENDED DECEMBER 31, 2015 [ ]TRANSITION REPORT PURSUANT TO SECTION 13 OR 15(d) OF THE SECURITIES EXCHANGE ACT OF 1934orFOR THE TRANSITION PERIOD FROM ___________ TO __________ COMMISSION FILE NUMBER 1-3551 EQT CORPORATION(Exact name of registrant as specified in its charter) PENNSYLVANIA(State or other jurisdiction of incorporation or organization) 25-0464690(IRS Employer Identification No.) 625 Liberty AvenuePittsburgh, Pennsylvania(Address of principal executive offices)15222(Zip Code) Registrant’s telephone number, including area code: (412) 553-5700 Securities registered pursuant to Section 12(b) of the Act: Title of each className of each exchange on which registeredCommon Stock, no par valueNew ...",1,1.0
12,"\n10-K\n1\nForm_10_K.htm\n\n\n\n\n\n\n\n\n\n\n \n\n\n\n\n\n\n \n\n\n\n \n\n\n\n \n\n\n\n\nUNITED STATES SECURITIES AND EXCHANGE\nCOMMISSION \n\nWashington, D.C. 20549 \n\n \n\nFORM 10-K \n\n\n\n\n \n \n \n \n \n \n \n \n \n \n \n\n \n ☑ \n \n \n \n \n \n ANNUAL REPORT PURSUANT TO SECTION 13 OR\n 15(d) OF THE SECURITIES EXCHANGE ACT OF 1934 \n \n \n\n\n\n\n\n\nFor the fiscal year ended December 31, 2016\n\nor \n\n\n\n\n \n \n \n \n \n \n \n \n \n \n \n\n \n o \n \n \n \n \n \n TRANSITION REPORT PURSUANT TO SECTION 13\n OR 15(d) OF THE SECURITIES EXCHANGE ACT OF 1934 \n \n \n \n\n\n\n\n\n\nFor the transition period from \n \nto \n \n\n \n\nCommission file number: 1-33615 \n\n \n\nConcho Resources Inc. \n\n(Exact name of registrant as specified in its charter) \n\n\n \n \n \n \n \n \n \n \n \n \n \n\n \n Delaware \n \n \n \n \n \n 76-081860...",1,1.0
43,"\n10-K/A\n1\na19-8685_110ka.htm\n10-K/A\n\n\n\n\n\n\n\n\n\n \n\n\nTable of Contents\n \nUNITED STATES\